## Dự đoán xâm nhập mạng bằng Mạng Neuron Tích chập (CNN)

### 1. Nhập các thư viện cần thiết

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import seaborn as sns
import matplotlib.pyplot as plt

### 2. Tải và khám phá dữ liệu

In [ ]:
file_path = 'D:/vhproj/intrusion-network/data/processed/demo_data.csv'
# Do tệp dữ liệu lớn, hãy xem xét sử dụng chunksize nếu bạn gặp vấn đề về bộ nhớ
df = pd.read_csv(file_path)
print(df.head())
print(df.info())
print(df['label'].value_counts())

### 3. Tiền xử lý dữ liệu

In [ ]:
# Tách các tính năng (X) và mục tiêu (y)
X = df.drop('label', axis=1)
y = df['label']

# Xác định các cột phân loại và số
# Dựa trên tên cột, 'Protocol Type' có vẻ là phân loại.
categorical_cols = ['Protocol Type']
numerical_cols = [col for col in X.columns if col not in categorical_cols]

# One-Hot Encode cho các cột phân loại
X = pd.get_dummies(X, columns=categorical_cols, drop_first=True)

# Mã hóa biến mục tiêu
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
num_classes = len(np.unique(y_encoded))

# Chia tỷ lệ các tính năng số
scaler = StandardScaler()
X[numerical_cols] = scaler.fit_transform(X[numerical_cols])

print("Số lượng tính năng sau khi mã hóa one-hot:", X.shape[1])
print("Số lượng lớp:", num_classes)

### 4. Định hình lại dữ liệu cho đầu vào CNN

In [ ]:
# CNN yêu cầu đầu vào có dạng hình ảnh (ví dụ: width, height, channels)
# Chúng ta sẽ định hình lại dữ liệu 1D của mình thành 2D.
n_features = X.shape[1]

# Tìm các yếu tố gần nhất để tạo thành một hình vuông hoặc hình chữ nhật
side = int(np.ceil(np.sqrt(n_features)))
padded_size = side * side

# Thêm đệm zero vào dữ liệu
X_padded = np.zeros((X.shape[0], padded_size))
X_padded[:, :n_features] = X.values

# Định hình lại thành một 'hình ảnh' 2D
X_reshaped = X_padded.reshape(-1, side, side, 1) # (samples, height, width, channels)

print("Hình dạng dữ liệu đã định hình lại:", X_reshaped.shape)

### 5. Chia dữ liệu thành tập huấn luyện và tập kiểm tra

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

### 6. Xây dựng mô hình CNN

In [ ]:
model = Sequential([
    # Lớp tích chập đầu tiên
    Conv2D(32, (3, 3), activation='relu', input_shape=X_train.shape[1:]),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    
    # Lớp tích chập thứ hai
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    
    # Làm phẳng và các lớp dày đặc
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax') # Lớp đầu ra
])

model.summary()

### 7. Biên dịch mô hình

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

### 8. Huấn luyện mô hình

In [ ]:
history = model.fit(X_train, y_train, 
                    epochs=20, 
                    batch_size=128, 
                    validation_split=0.2, 
                    verbose=1)

### 9. Đánh giá mô hình

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f'Độ chính xác trên tập kiểm tra: {accuracy:.4f}')
print(f'Tổn thất trên tập kiểm tra: {loss:.4f}')

### 10. Báo cáo phân loại và Ma trận nhầm lẫn

In [ ]:
# Dự đoán
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

# Báo cáo phân loại
print("Báo cáo phân loại:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

# Ma trận nhầm lẫn
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.title('Ma trận nhầm lẫn')
plt.ylabel('Nhãn thực tế')
plt.xlabel('Nhãn dự đoán')
plt.show()